In [28]:
reg_a = 0
reg_b = 0
reg_c = 0
program = []
lc = 0

with open('example2.txt') as f:
    for line in f.readlines():
        if lc == 0:
            reg_a = int(line.split(":")[1].strip())
        elif lc == 1:
            reg_b = int(line.split(":")[1].strip())
        elif lc == 2:
            reg_b = int(line.split(":")[1].strip())
        elif lc == 4:
            program = [int(n) for n in line.split(":")[1].strip().split(",")]
        lc += 1

In [8]:
class Computer():
    def opcode_to_func(self, opcode):
        return {
            0: self.adv,
            1: self.bxl,
            2: self.bst,
            3: self.jnz,
            4: self.bxc,
            5: self.out,
            6: self.bdv,
            7: self.cdv
        }[opcode]

    def __init__(self, a, b, c, program):
        self.reg_a = a
        self.reg_b = b
        self.reg_c = c
        self.program = program
        self.pc = 0
        self.output = []

    def run(self):
        while self.pc < len(self.program):
            try: 
                self.opcode_to_func(self.program[self.pc])()
            except IndexError:
                break

    def get_combo_op(self, op):
        return {
            0: 0,
            1: 1,
            2: 2,
            3: 3,
            4: self.reg_a,
            5: self.reg_b,
            6: self.reg_c
        }[op]

    def adv(self):
        operand = self.program[self.pc+1]
        numerator = self.reg_a
        divisor = 2**self.get_combo_op(operand)
        self.reg_a = numerator // divisor
        self.pc += 2

    def bxl(self):
        operand = self.program[self.pc+1]
        self.reg_b = self.reg_b ^ operand
        self.pc += 2

    def bst(self):
        operand = self.program[self.pc+1]
        combo_op = self.get_combo_op(operand)
        self.reg_b = combo_op % 8
        self.pc += 2

    def jnz(self):
        if self.reg_a == 0:
            self.pc += 1
            return
        operand = self.program[self.pc+1]
        self.pc = operand

    def bxc(self):
        self.reg_b = self.reg_b ^ self.reg_c
        self.pc += 2  # even though we don't use the operand, we still need to increment the pc

    def out(self):
        operand = self.program[self.pc+1]
        self.output.append(self.get_combo_op(operand) % 8)
        self.pc += 2

    def bdv(self):
        operand = self.program[self.pc+1]
        numerator = self.reg_a
        divisor = 2**self.get_combo_op(operand)
        self.reg_b = numerator // divisor
        self.pc += 2

    def cdv(self):
        operand = self.program[self.pc+1]
        numerator = self.reg_a
        divisor = 2**self.get_combo_op(operand)
        self.reg_c = numerator // divisor
        self.pc += 2

    def __str__(self):
        return f"""
Computer(a={self.reg_a}, b={self.reg_b}, c={self.reg_c}, program={self.program}, pc={self.pc})
Output: {",".join([str(n) for n in self.output])}
        """

    def __repr__(self):
        return str(self)

In [30]:
# for i in range(23066730086400, 230667300964000, 8^3):
for w in range(0, 64, 8):
    for x in range(0, 64, 8):
        for z in range(0, 3119, 1):
            for y in range(0, 32, 1):
                c = Computer(reg_a, reg_b, reg_c, program)
                c.reg_a = w*x*y*z
                c.run()

                if c.output == program:
                    print(f"Found! {w}, {x}, {y}, {z}")
                    raise SystemExit